In [1]:

import pandas as pd
from dateutil.relativedelta import *
from TimeBasedCV import TimeBasedCV


In [2]:
transactions = pd.read_parquet('../data/HM_parquet/transactions_train.parquet')
customers = pd.read_parquet('../data/HM_parquet/customers.parquet')
articles = pd.read_parquet('../data/HM_parquet/articles.parquet')

tscv = TimeBasedCV(freq='days')
    

# implicit.ALS

In [3]:
from model.ImplicitALS import ImplicitALS as als

model = als()

## 測試一個月的資料

In [4]:
# 測試一個月的資料
train_one_month = pd.read_parquet('data/train_one_month.parquet')
val_one_parquet = pd.read_parquet('data/val_one_month.parquet')

train_one_month = model.data_preprocess(train_one_month,customers,articles)
val_one_parquet = model.data_preprocess(val_one_parquet,customers,articles)

train_period, val_period, stride = 30, 7, 30
scores_one_month = pd.DataFrame(columns=["train_period","val_period","stride"])

one_fold_scores = model.train_ALS(train_one_month, val_one_parquet, train_period, val_period, stride,start_val=0)
scores_one_month = pd.concat([scores_one_month,one_fold_scores], axis=0 ,ignore_index=True)

scores_one_month


{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 0, 'factors': 25, 'iterations': 3, 'regularization': 0.01, 'map12': 0.003136429035660723}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 0, 'factors': 25, 'iterations': 10, 'regularization': 0.01, 'map12': 0.004087349085210472}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 0, 'factors': 25, 'iterations': 20, 'regularization': 0.01, 'map12': 0.004186328448466547}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 0, 'factors': 50, 'iterations': 3, 'regularization': 0.01, 'map12': 0.0037242955926242703}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 0, 'factors': 50, 'iterations': 10, 'regularization': 0.01, 'map12': 0.004775751555119398}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 0, 'factors': 50, 'iterations': 20, 'regularization': 0.01, 'map12': 0.004815271571309318}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 

train_period val_period stride  start_val  factors  iterations  \
0            30          7     30        0.0     25.0         3.0   
1            30          7     30        0.0     25.0        10.0   
2            30          7     30        0.0     25.0        20.0   
3            30          7     30        0.0     50.0         3.0   
4            30          7     30        0.0     50.0        10.0   
5            30          7     30        0.0     50.0        20.0   
6            30          7     30        0.0    100.0         3.0   
7            30          7     30        0.0    100.0        10.0   
8            30          7     30        0.0    100.0        20.0   
9            30          7     30        0.0    200.0         3.0   
10           30          7     30        0.0    200.0        10.0   
11           30          7     30        0.0    200.0        20.0   

    regularization     map12  
0             0.01  0.003136  
1             0.01  0.004087  
2             0.01  0.004186  
3             0.01  0.003724  
4             0.01  0.004776  
5             0.01  0.004815  
6             0.01  0.004235  
7             0.01  0.005260  
8             0.01  0.005238  
9             0.01  0.004667  
10            0.01  0.005123  
11            0.01  0.005061

In [5]:
scores_one_month

train_period val_period stride  start_val  factors  iterations  \
0            30          7     30        0.0     25.0         3.0   
1            30          7     30        0.0     25.0        10.0   
2            30          7     30        0.0     25.0        20.0   
3            30          7     30        0.0     50.0         3.0   
4            30          7     30        0.0     50.0        10.0   
5            30          7     30        0.0     50.0        20.0   
6            30          7     30        0.0    100.0         3.0   
7            30          7     30        0.0    100.0        10.0   
8            30          7     30        0.0    100.0        20.0   
9            30          7     30        0.0    200.0         3.0   
10           30          7     30        0.0    200.0        10.0   
11           30          7     30        0.0    200.0        20.0   

    regularization     map12  
0             0.01  0.003136  
1             0.01  0.004087  
2             0.01  0.004186  
3             0.01  0.003724  
4             0.01  0.004776  
5             0.01  0.004815  
6             0.01  0.004235  
7             0.01  0.005260  
8             0.01  0.005238  
9             0.01  0.004667  
10            0.01  0.005123  
11            0.01  0.005061

## 分別取不同時間段

In [5]:

import pandas as pd
from dateutil.relativedelta import *
from model.TimeBasedCV import TimeBasedCV
from model.ImplicitALS import ImplicitALS as als

model = als()


In [6]:
transactions = pd.read_parquet('data\\HM_parquet\\transactions_train.parquet')
customers = pd.read_parquet('data\\HM_parquet\\customers.parquet')
articles = pd.read_parquet('data\\HM_parquet\\articles.parquet')

tscv = TimeBasedCV(freq='days')
    

In [7]:
transactions = model.data_preprocess(transactions,customers,articles)

In [8]:
# 做 time based split 30天
train_period, val_period, stride = 30, 7, 30
index_output = tscv.split(transactions, date_column='t_dat', train_period=train_period, val_period=val_period, stride=stride,show_progress=True)

# 做 time based CV
scores = pd.DataFrame(columns=["train_period","val_period","stride"])

for train_index, val_index in index_output:
    train_data = transactions.loc[train_index]
    val_data = transactions.loc[val_index]
    # 取得val開始日期
    val_data.reset_index(inplace=True, drop=True)
    start_val = val_data['t_dat'][0]
    # 呼叫訓練模型的function
    one_fold_scores = model.train_ALS(train_data, val_data, train_period, val_period, stride, start_val)
    scores = pd.concat([scores,one_fold_scores], axis=0 ,ignore_index=True)

scores.to_parquet('model/params/params_stride30/implicit_ALS_train30.parquet')

Train period: 2020-08-16 00:00:00 - 2020-09-15 00:00:00 , val period 2020-09-15 00:00:00 - 2020-09-22 00:00:00 # train records 1123728 , # val records 233498
Train period: 2020-07-17 00:00:00 - 2020-08-16 00:00:00 , val period 2020-08-16 00:00:00 - 2020-08-23 00:00:00 # train records 1275928 , # val records 234159
Train period: 2020-06-17 00:00:00 - 2020-07-17 00:00:00 , val period 2020-07-17 00:00:00 - 2020-07-24 00:00:00 # train records 1787814 , # val records 284020
Train period: 2020-05-18 00:00:00 - 2020-06-17 00:00:00 , val period 2020-06-17 00:00:00 - 2020-06-24 00:00:00 # train records 1455684 , # val records 549443
Train period: 2020-04-18 00:00:00 - 2020-05-18 00:00:00 , val period 2020-05-18 00:00:00 - 2020-05-25 00:00:00 # train records 1105288 , # val records 374636
Train period: 2020-03-19 00:00:00 - 2020-04-18 00:00:00 , val period 2020-04-18 00:00:00 - 2020-04-25 00:00:00 # train records 1249538 , # val records 253001
Train period: 2020-02-18 00:00:00 - 2020-03-19 00:00

C:\Users\Student\anaconda3\envs\Python38_20211231\lib\site-packages\implicit\utils.py:31: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': Timestamp('2020-09-15 00:00:00'), 'factors': 25, 'iterations': 3, 'regularization': 0.01, 'map12': 0.0032450649843469392}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': Timestamp('2020-09-15 00:00:00'), 'factors': 25, 'iterations': 10, 'regularization': 0.01, 'map12': 0.004237436784330226}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': Timestamp('2020-09-15 00:00:00'), 'factors': 25, 'iterations': 20, 'regularization': 0.01, 'map12': 0.0043506078569621725}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': Timestamp('2020-09-15 00:00:00'), 'factors': 50, 'iterations': 3, 'regularization': 0.01, 'map12': 0.003901553741591228}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': Timestamp('2020-09-15 00:00:00'), 'factors': 50, 'iterations': 10, 'regularization': 0.01, 'map12': 0.005025365811129218}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 

In [9]:
# 做 time based split 60天
train_period, val_period, stride = 60, 7, 30
index_output = tscv.split(transactions, date_column='t_dat', train_period=train_period, val_period=val_period, stride=stride,show_progress=True)

# 做 time based CV
scores = pd.DataFrame(columns=["train_period","val_period","stride"])

for train_index, val_index in index_output:
    train_data = transactions.loc[train_index]
    val_data = transactions.loc[val_index]
    # 取得val開始日期
    val_data.reset_index(inplace=True, drop=True)
    start_val = val_data['t_dat'][0]
    # 呼叫訓練模型的function
    one_fold_scores = model.train_ALS(train_data, val_data, train_period, val_period, stride, start_val)
    scores = pd.concat([scores,one_fold_scores], axis=0 ,ignore_index=True)

# this is the average accuracy over all folds
# average_r2score = np.mean(scores)

scores.to_parquet('model/params/params_stride30/implicit_ALS_train60.parquet')

Train period: 2020-07-17 00:00:00 - 2020-09-15 00:00:00 , val period 2020-09-15 00:00:00 - 2020-09-22 00:00:00 # train records 2399656 , # val records 233498
Train period: 2020-06-17 00:00:00 - 2020-08-16 00:00:00 , val period 2020-08-16 00:00:00 - 2020-08-23 00:00:00 # train records 3063742 , # val records 234159
Train period: 2020-05-18 00:00:00 - 2020-07-17 00:00:00 , val period 2020-07-17 00:00:00 - 2020-07-24 00:00:00 # train records 3243498 , # val records 284020
Train period: 2020-04-18 00:00:00 - 2020-06-17 00:00:00 , val period 2020-06-17 00:00:00 - 2020-06-24 00:00:00 # train records 2560972 , # val records 549443
Train period: 2020-03-19 00:00:00 - 2020-05-18 00:00:00 , val period 2020-05-18 00:00:00 - 2020-05-25 00:00:00 # train records 2354826 , # val records 374636
Train period: 2020-02-18 00:00:00 - 2020-04-18 00:00:00 , val period 2020-04-18 00:00:00 - 2020-04-25 00:00:00 # train records 2306996 , # val records 253001
Train period: 2020-01-19 00:00:00 - 2020-03-19 00:00

In [10]:
# 做 time based split 90天
train_period, val_period, stride = 90, 7, 30
index_output = tscv.split(transactions, date_column='t_dat', train_period=train_period, val_period=val_period, stride=stride,show_progress=True)

# 做 time based CV
scores = pd.DataFrame(columns=["train_period","val_period","stride"])

for train_index, val_index in index_output:
    train_data = transactions.loc[train_index]
    val_data = transactions.loc[val_index]
    # 取得val開始日期
    val_data.reset_index(inplace=True, drop=True)
    start_val = val_data['t_dat'][0]
    # 呼叫訓練模型的function
    one_fold_scores = model.train_ALS(train_data, val_data, train_period, val_period, stride, start_val)
    scores = pd.concat([scores,one_fold_scores], axis=0 ,ignore_index=True)

# this is the average accuracy over all folds
# average_r2score = np.mean(scores)

scores.to_parquet('model/params/params_stride30/implicit_ALS_train90.parquet')

Train period: 2020-06-17 00:00:00 - 2020-09-15 00:00:00 , val period 2020-09-15 00:00:00 - 2020-09-22 00:00:00 # train records 4187470 , # val records 233498
Train period: 2020-05-18 00:00:00 - 2020-08-16 00:00:00 , val period 2020-08-16 00:00:00 - 2020-08-23 00:00:00 # train records 4519426 , # val records 234159
Train period: 2020-04-18 00:00:00 - 2020-07-17 00:00:00 , val period 2020-07-17 00:00:00 - 2020-07-24 00:00:00 # train records 4348786 , # val records 284020
Train period: 2020-03-19 00:00:00 - 2020-06-17 00:00:00 , val period 2020-06-17 00:00:00 - 2020-06-24 00:00:00 # train records 3810510 , # val records 549443
Train period: 2020-02-18 00:00:00 - 2020-05-18 00:00:00 , val period 2020-05-18 00:00:00 - 2020-05-25 00:00:00 # train records 3412284 , # val records 374636
Train period: 2020-01-19 00:00:00 - 2020-04-18 00:00:00 , val period 2020-04-18 00:00:00 - 2020-04-25 00:00:00 # train records 3346979 , # val records 253001
Train period: 2019-12-20 00:00:00 - 2020-03-19 00:00

In [11]:
# 做 time based split 180天
train_period, val_period, stride = 180, 7, 30
index_output = tscv.split(transactions, date_column='t_dat', train_period=train_period, val_period=val_period, stride=stride,show_progress=True)

# 做 time based CV
scores = pd.DataFrame(columns=["train_period","val_period","stride"])

for train_index, val_index in index_output:
    train_data = transactions.loc[train_index]
    val_data = transactions.loc[val_index]
    # 取得val開始日期
    val_data.reset_index(inplace=True, drop=True)
    start_val = val_data['t_dat'][0]
    # 呼叫訓練模型的function
    one_fold_scores = model.train_ALS(train_data, val_data, train_period, val_period, stride, start_val)
    scores = pd.concat([scores,one_fold_scores], axis=0 ,ignore_index=True)

# this is the average accuracy over all folds
# average_r2score = np.mean(scores)

scores.to_parquet('model/params/params_stride30/implicit_ALS_train180.parquet')

Train period: 2020-03-19 00:00:00 - 2020-09-15 00:00:00 , val period 2020-09-15 00:00:00 - 2020-09-22 00:00:00 # train records 7997980 , # val records 233498
Train period: 2020-02-18 00:00:00 - 2020-08-16 00:00:00 , val period 2020-08-16 00:00:00 - 2020-08-23 00:00:00 # train records 7931710 , # val records 234159
Train period: 2020-01-19 00:00:00 - 2020-07-17 00:00:00 , val period 2020-07-17 00:00:00 - 2020-07-24 00:00:00 # train records 7695765 , # val records 284020
Train period: 2019-12-20 00:00:00 - 2020-06-17 00:00:00 , val period 2020-06-17 00:00:00 - 2020-06-24 00:00:00 # train records 6979219 , # val records 549443
Train period: 2019-11-20 00:00:00 - 2020-05-18 00:00:00 , val period 2020-05-18 00:00:00 - 2020-05-25 00:00:00 # train records 6775472 , # val records 374636
Train period: 2019-10-21 00:00:00 - 2020-04-18 00:00:00 , val period 2020-04-18 00:00:00 - 2020-04-25 00:00:00 # train records 6656322 , # val records 253001
Train period: 2019-09-21 00:00:00 - 2020-03-19 00:00

In [12]:
# 做 time based split 270天
train_period, val_period, stride = 270, 7, 30
index_output = tscv.split(transactions, date_column='t_dat', train_period=train_period, val_period=val_period, stride=stride,show_progress=True)

# 做 time based CV
scores = pd.DataFrame(columns=["train_period","val_period","stride"])

for train_index, val_index in index_output:
    train_data = transactions.loc[train_index]
    val_data = transactions.loc[val_index]
    # 取得val開始日期
    val_data.reset_index(inplace=True, drop=True)
    start_val = val_data['t_dat'][0]
    # 呼叫訓練模型的function
    one_fold_scores = model.train_ALS(train_data, val_data, train_period, val_period, stride, start_val)
    scores = pd.concat([scores,one_fold_scores], axis=0 ,ignore_index=True)

# this is the average accuracy over all folds
# average_r2score = np.mean(scores)

scores.to_parquet('model/params/params_stride30/implicit_ALS_train270.parquet')

Train period: 2019-12-20 00:00:00 - 2020-09-15 00:00:00 , val period 2020-09-15 00:00:00 - 2020-09-22 00:00:00 # train records 11166689 , # val records 233498
Train period: 2019-11-20 00:00:00 - 2020-08-16 00:00:00 , val period 2020-08-16 00:00:00 - 2020-08-23 00:00:00 # train records 11294898 , # val records 234159
Train period: 2019-10-21 00:00:00 - 2020-07-17 00:00:00 , val period 2020-07-17 00:00:00 - 2020-07-24 00:00:00 # train records 11005108 , # val records 284020
Train period: 2019-09-21 00:00:00 - 2020-06-17 00:00:00 , val period 2020-06-17 00:00:00 - 2020-06-24 00:00:00 # train records 10499081 , # val records 549443
Train period: 2019-08-22 00:00:00 - 2020-05-18 00:00:00 , val period 2020-05-18 00:00:00 - 2020-05-25 00:00:00 # train records 10098648 , # val records 374636
Train period: 2019-07-23 00:00:00 - 2020-04-18 00:00:00 , val period 2020-04-18 00:00:00 - 2020-04-25 00:00:00 # train records 10482190 , # val records 253001
Train period: 2019-06-23 00:00:00 - 2020-03-19

In [13]:
# 做 time based split 360天
train_period, val_period, stride = 360, 7, 30
index_output = tscv.split(transactions, date_column='t_dat', train_period=train_period, val_period=val_period, stride=stride,show_progress=True)

# 做 time based CV
scores = pd.DataFrame(columns=["train_period","val_period","stride"])

for train_index, val_index in index_output:
    train_data = transactions.loc[train_index]
    val_data = transactions.loc[val_index]
    # 取得val開始日期
    val_data.reset_index(inplace=True, drop=True)
    start_val = val_data['t_dat'][0]
    # 呼叫訓練模型的function
    one_fold_scores = model.train_ALS(train_data, val_data, train_period, val_period, stride, start_val)
    scores = pd.concat([scores,one_fold_scores], axis=0 ,ignore_index=True)

# this is the average accuracy over all folds
# average_r2score = np.mean(scores)

scores.to_parquet('model/params/params_stride30/implicit_ALS_train360.parquet')

Train period: 2019-09-21 00:00:00 - 2020-09-15 00:00:00 , val period 2020-09-15 00:00:00 - 2020-09-22 00:00:00 # train records 14686551 , # val records 233498
Train period: 2019-08-22 00:00:00 - 2020-08-16 00:00:00 , val period 2020-08-16 00:00:00 - 2020-08-23 00:00:00 # train records 14618074 , # val records 234159
Train period: 2019-07-23 00:00:00 - 2020-07-17 00:00:00 , val period 2020-07-17 00:00:00 - 2020-07-24 00:00:00 # train records 14830976 , # val records 284020
Train period: 2019-06-23 00:00:00 - 2020-06-17 00:00:00 , val period 2020-06-17 00:00:00 - 2020-06-24 00:00:00 # train records 14921258 , # val records 549443
Train period: 2019-05-24 00:00:00 - 2020-05-18 00:00:00 , val period 2020-05-18 00:00:00 - 2020-05-25 00:00:00 # train records 15181996 , # val records 374636
Train period: 2019-04-24 00:00:00 - 2020-04-18 00:00:00 , val period 2020-04-18 00:00:00 - 2020-04-25 00:00:00 # train records 15568264 , # val records 253001
Train period: 2019-03-25 00:00:00 - 2020-03-19

In [14]:
# 合併所有dataframe
periods = [30,60,90,180,270,360]
total_scores = pd.DataFrame()

for period in periods:
    perios_socres = pd.read_parquet(f"model/params/params_stride30/implicit_ALS_train{period}.parquet")
    total_scores = pd.concat([total_scores,perios_socres],axis=0,ignore_index=True)
    
total_scores

train_period  val_period  stride  start_val  factors  iterations  \
0               30           7      30 2020-09-15     25.0         3.0   
1               30           7      30 2020-09-15     25.0        10.0   
2               30           7      30 2020-09-15     25.0        20.0   
3               30           7      30 2020-09-15     50.0         3.0   
4               30           7      30 2020-09-15     50.0        10.0   
...            ...         ...     ...        ...      ...         ...   
1399           360           7      30 2019-09-21    100.0        10.0   
1400           360           7      30 2019-09-21    100.0        20.0   
1401           360           7      30 2019-09-21    200.0         3.0   
1402           360           7      30 2019-09-21    200.0        10.0   
1403           360           7      30 2019-09-21    200.0        20.0   

      regularization     map12  
0               0.01  0.003245  
1               0.01  0.004237  
2               0.01  0.004351  
3               0.01  0.003902  
4               0.01  0.005025  
...              ...       ...  
1399            0.01  0.003802  
1400            0.01  0.003714  
1401            0.01  0.003584  
1402            0.01  0.003621  
1403            0.01  0.003610  

[1404 rows x 8 columns]

In [17]:
# 看最佳的參數組合
total_scores.groupby(['train_period','factors','iterations','regularization']).mean('map12').sort_values('map12',ascending=False)

val_period  stride     map12
train_period factors iterations regularization                              
60           200.0   10.0       0.01                   7.0    30.0  0.004955
90           200.0   10.0       0.01                   7.0    30.0  0.004901
60           200.0   20.0       0.01                   7.0    30.0  0.004888
90           200.0   20.0       0.01                   7.0    30.0  0.004815
60           100.0   10.0       0.01                   7.0    30.0  0.004628
...                                                    ...     ...       ...
90           25.0    3.0        0.01                   7.0    30.0  0.002914
180          25.0    3.0        0.01                   7.0    30.0  0.002893
270          25.0    3.0        0.01                   7.0    30.0  0.002868
60           25.0    3.0        0.01                   7.0    30.0  0.002848
30           25.0    3.0        0.01                   7.0    30.0  0.002482

[72 rows x 3 columns]

## 彙整

In [1]:

import pandas as pd
from dateutil.relativedelta import *
from model.TimeBasedCV import TimeBasedCV

transactions = pd.read_parquet('data\\HM_parquet\\transactions_train.parquet')
customers = pd.read_parquet('data\\HM_parquet\\customers.parquet')
articles = pd.read_parquet('data\\HM_parquet\\articles.parquet')

tscv = TimeBasedCV(freq='days')

In [9]:

# 測試將函數傳入函數
def test(model,train_data,val_data,train_period,stride):
    val_period = 7
    scores_one_month = pd.DataFrame(columns=["train_period","val_period","stride"])

    one_fold_scores = model(train_data, val_data, train_period, val_period, stride,start_val=0)
    scores_one_month = pd.concat([scores_one_month,one_fold_scores], axis=0 ,ignore_index=True)
    return scores_one_month

model = als()

# 測試一個月的資料
# transactions = pd.read_parquet('data\\HM_parquet\\transactions_train.parquet')
train_one_month = pd.read_parquet('data/train_one_month.parquet')
val_one_parquet = pd.read_parquet('data/val_one_month.parquet')

train_one_month = model.data_preprocess(train_one_month,customers,articles)
val_one_parquet = model.data_preprocess(val_one_parquet,customers,articles)

scores_one_month = test(model.train_ALS,train_one_month,val_one_parquet,30,30)
scores_one_month

{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 0, 'factors': 25, 'iterations': 3, 'regularization': 0.01, 'map12': 0.003136429035660723}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 0, 'factors': 25, 'iterations': 10, 'regularization': 0.01, 'map12': 0.004087349085210472}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 0, 'factors': 25, 'iterations': 20, 'regularization': 0.01, 'map12': 0.004186328448466547}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 0, 'factors': 50, 'iterations': 3, 'regularization': 0.01, 'map12': 0.0037242955926242703}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 0, 'factors': 50, 'iterations': 10, 'regularization': 0.01, 'map12': 0.004775751555119398}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 0, 'factors': 50, 'iterations': 20, 'regularization': 0.01, 'map12': 0.004815271571309318}
{'train_period': 30, 'val_period': 7, 'stride': 30, 'start_val': 

train_period val_period stride  start_val  factors  iterations  \
0            30          7     30        0.0     25.0         3.0   
1            30          7     30        0.0     25.0        10.0   
2            30          7     30        0.0     25.0        20.0   
3            30          7     30        0.0     50.0         3.0   
4            30          7     30        0.0     50.0        10.0   
5            30          7     30        0.0     50.0        20.0   
6            30          7     30        0.0    100.0         3.0   
7            30          7     30        0.0    100.0        10.0   
8            30          7     30        0.0    100.0        20.0   
9            30          7     30        0.0    200.0         3.0   
10           30          7     30        0.0    200.0        10.0   
11           30          7     30        0.0    200.0        20.0   

    regularization     map12  
0             0.01  0.003136  
1             0.01  0.004087  
2             0.01  0.004186  
3             0.01  0.003724  
4             0.01  0.004776  
5             0.01  0.004815  
6             0.01  0.004235  
7             0.01  0.005260  
8             0.01  0.005238  
9             0.01  0.004667  
10            0.01  0.005123  
11            0.01  0.005061

In [2]:

def time_base_model(model,data,date_column,train_period,stride):
    # time based split
    val_period=7
    index_output = tscv.split(data, date_column, train_period, val_period, stride, show_progress=False)

    # 做 time based CV
    scores = pd.DataFrame(columns=["train_period","val_period","stride"])

    for train_index, val_index in index_output:
        train_data = transactions.loc[train_index]
        val_data = transactions.loc[val_index]
        # 取得val開始日期
        val_data.reset_index(inplace=True, drop=True)
        start_val = val_data[date_column][0]
        # 呼叫訓練模型的function
        one_fold_scores = model(train_data, val_data, train_period, val_period, stride, start_val)
        scores = pd.concat([scores,one_fold_scores], axis=0 ,ignore_index=True)
        
    return scores

In [3]:
# 用迴圈跑不同時間段
from model.ImplicitALS import ImplicitALS as als

model = als()

data = model.data_preprocess(transactions,customers,articles)
date_column='t_dat'
train_periods = [30,60,90,180,270,360]
stride = 30

# 做 time based split
for train_period in train_periods:
    one_period_scores = time_base_model(model,transactions,date_column,train_period,stride)
    one_period_scores.to_parquet(f'model/params/params_stride30/implicit_ALS_train{train_period}.parquet')




MemoryError: Unable to allocate 30.3 MiB for an array with shape (31788324,) and data type bool

In [ ]:
666666666